<a href="https://colab.research.google.com/github/MartinYTSo/File_Finder/blob/main/file_screener_src.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** THE PYTHON SOURCE CODE FOR THE FILE SCREENER. 
FRAMEWORK USED IS PYSIDE2(QT FOR PYTHON) **

In [ ]:
# This Desktop App is used as an automatic online voting system

import os, sys, time, datetime, re, requests, zipfile, json, random, logging, trace
from urllib.request import urlopen as ureq
from base64 import b64encode
from sys import platform
from queue import Queue
import threading, concurrent.futures
import pandas as pd
from openpyxl import load_workbook
import numpy as np
from bs4 import BeautifulSoup as bs
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from deathbycaptcha import deathbycaptcha

import win32com.client
from docx import Document


# Using selenium
import selenium
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox, Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.alert import Alert

# For the gui interface widgets
import PySide2
from PySide2.QtWidgets import (
    QApplication, QDialog, QLineEdit, QPushButton, QTextEdit, QMenu, QGroupBox, QCheckBox, QComboBox, QFormLayout,
    QListWidget, QVBoxLayout, QHBoxLayout, QGridLayout, QMainWindow, QWidget, QAction, QLabel, QProgressBar, QScrollArea,
    QRadioButton, QFileDialog, QMessageBox     # Added these for the file file finder functionality.
    )
from PySide2.QtCore import Slot, SIGNAL, QThread, QThreadPool, QRunnable, QObject, Signal
from PySide2.QtGui import QIcon, QTextCursor
from ui_mainwindow import Ui_MainWindow

# Accessing files in directories
if getattr(sys, 'frozen', False):
    # running in a bundled form
    base_dir = sys._MEIPASS # pylint: disable=no-member
else:
    # running normally
    base_dir = os.path.dirname(os.path.abspath(__file__))

# Locating helper files in the current working directory
# icon_file_path = os.path.join(base_dir, 'icon.png')

# # number of tracking numbers to fetch from is N
global N
N = 3




# The solution is to use a thread-safe object (like a Python Queue.Queue)
# to mediate the transfer of information. I've attached some sample code below which
# redirects stdout to a Python Queue. This Queue is read by a QThread, which emits the
# contents to the main thread through Qt's signal/slot mechanism (emitting signals is thread-safe).
# The main thread then writes the text to a text edit.
# The new Stream Object which replaces the default stream associated with sys.stdout
# This object just puts data in a queue!
class WriteStream(object):
    def __init__(self, queue):
        self.queue = queue

    def write(self, text):
        self.queue.put(text)



# A QObject (to be run in a QThread) which sits waiting for data to come through a Queue.Queue().
# It blocks until data is available, and once it has got something from the queue, it sends
# it to the "MainThread" by emitting a Qt Signal 
class MyReceiver(QObject):
       mysignal = Signal(str)

       def __init__(self, queue, *args, **kwargs):
           QObject.__init__(self, *args, **kwargs)
           self.queue = queue

       @Slot()
       def run(self):
           while True:
               text = self.queue.get()
               self.mysignal.emit(text)
               




# Related to PySide2, QThread class is commonly used for splitting of tasks into multiple threads to increase the 
# speed of the GUI application, because a large number of tasks in one thread make the application slow and frozen.
# As described above, the greatest benefit between the PySide2 QThread class and the python threading module from stdlib 
# is the support of sending and emitting signals. These signals can be a string with text, lists with variables, tuples, 
# integers, other python types. You can change the color of these apps, insert or set or append texts, draw, paint and have 
# many other features to create GUI application fast and flexible. 
# We also don't need to use the time.sleep(int) because there's a sleep method on all QThread objects which we can use.
class ThreadingClass(QThread):
    def __init__(self):
        super(ThreadingClass, self).__init__()

    # Our Confirmation BOT Method(USING THE WEB BROWSER)
    def startAutomation(self):
        try:
            # print('Hello Cyber Overlord!')
            # print(f"You chose the search method: {search_method}")
            # print(f"You are searching the path: {file_search_path}")
            # print(f"You are searching for the key_word: {search_keyword}")
            # # self.sleep(6)

            # Traverse the directory
            directories_with_keyword = []
            for path, currentDirectory, files in os.walk(file_search_path):
                for file in files:
                    # print(file)
                    # Operate on .docx files
                    if file.endswith('.docx') and not file.startswith('~'):
                        # print(f"Found a docx file: {file}, searching file for keyword")

                        # Working with docx files
                        filename = os.path.join(path, f"{file}")
                        doc_obj = Document(filename)
                        regex = re.compile(f"{search_keyword}".lower())
                        # print(len(doc_obj.tables))
                        for p in doc_obj.paragraphs: 
                            if regex.search(p.text.lower()):
                                # print('Match found(1)!')
                                if filename in directories_with_keyword:
                                    pass
                                else:
                                    directories_with_keyword.append(filename)
                                inline = p.runs 
                                # Loop added to work with runs (strings with same style) 
                                for i in range(len(inline)):
                                    # print(inline[i].text)
                                    if regex.search(inline[i].text):
                                        # print('Match found(2)!')
                                        pass
                                    else:
                                        # print('No match found(2)!')
                                        pass
                            else:
                                # print('No match found(1)!')
                                pass

                        for table in doc_obj.tables: 
                            for row in table.rows: 
                                for cell in row.cells: 
                                    if regex.search(p.text.lower()):
                                        # print('Match found(1)!')
                                        if filename in directories_with_keyword:
                                            pass
                                        else:
                                            directories_with_keyword.append(filename) 
                                        inline = p.runs 
                                        # Loop added to work with runs (strings with same style) 
                                        for i in range(len(inline)):
                                            # print(inline[i].text)
                                            if regex.search(inline[i].text):
                                                # print('Match found(2)!') 
                                                pass
                                            else:
                                                # print('No match found(2)!')
                                                pass
                                    else:
                                        # print('No match found(1)!')
                                        pass

                    # # Operate on .doc files
                    # elif file.endswith('.doc') and not file.startswith('~'):
                    #     # print(f"Found doc file: {file}, searching file for keyword..")
                    #     # Working with doc files
                    #     ourFile_file_path = os.path.join(path, f'{file}')
                    #     word = win32com.client.Dispatch("Word.Application")
                    #     word.visible = False
                    #     wb = word.Documents.Open(ourFile_file_path)
                    #     doc = word.ActiveDocument
                    #     textString = doc.Range().Text
                    #     # print(textString)

                    #     # regex1 = re.compile(r"testing One one".lower())
                    #     regex1 = re.compile(f"{search_keyword}".lower())
                    #     # print(regex1.search(textString.lower()))
                    #     if regex1.search(textString.lower()):
                    #         # print("Keyword found")
                    #         if ourFile_file_path in directories_with_keyword:
                    #             pass
                    #         else:
                    #             directories_with_keyword.append(ourFile_file_path)

                    else:
                        pass

            print(f"The following directory or directories has the keyword: {search_keyword}")
            for directory_with_keyword in directories_with_keyword:
                print(f"Directory {directory_with_keyword}")



            # keyWordString_found = f"Keywordd: {search_keyword}, found in file .."
            keyWordString_found = f"Directory(s) with keyword: {directories_with_keyword}"



            # print('Operation successfully carried out on this operation!')
            # self.emit(SIGNAL('update_alert(QString)'), keyWordString_found)
            self.emit(SIGNAL('update_progress()'))


        except Exception as e:
            print(e)
            # Update our Progress Bar
            self.emit(SIGNAL('update_progress()'))
            pass

        return 'Operation complete'


    # Email Analysis function
    def run(self):
        # print(proxy_usage)
        # print("\n")
        print("Application Started!!")
        
        # Introduce debug waiter here
        # time.sleep(600)
        
        # Without threads
        # for link in (links_feeder_list):
        # # for username, password, proxy in zip(username_feeder_list, password_feeder_list, proxy_feeder_list):
        #     # self.emit(SIGNAL('update_progress(QString)'), "Email Analysis Completed!!!")
        #     startAutomation(userPass)
        #     # yahoomail_creator(username, password, proxy)
        #     self.emit(SIGNAL('update_progress()'))
        #     self.sleep(3)

        # *************************************************
        # # Making use of threading module
        # start_time = time.perf_counter()
        # threads = []

        # for link in links_feeder_list:
        # # for username, password, proxy in zip(username_feeder_list, password_feeder_list, proxy_feeder_list):
        #     t = threading.Thread(target=self.startAutomation, args=(link,))
        #     # t = threading.Thread(target=startAutomation, args=(username, password, proxy,))
        #     t.start()
        #     threads.append(t)

        # for thread in threads:
        #     thread.join()
        #     self.sleep(3)

        # *************************************************
        # Then making use of the threading pool executor
        # Using threadpool
        # if thread_count <= 0:
        #     num_ofThreads = 3
        # else:
        #     num_ofThreads = thread_count

        if thread_count == '':
            num_ofThreads = 1   # using my default number of threads
            # print('Using default max number of threads: ',  num_ofThreads)

        else:
            num_ofThreads = thread_count
            # print('Using max number of threads: ',  num_ofThreads)

        start_time = time.perf_counter()

        with concurrent.futures.ThreadPoolExecutor(int(num_ofThreads)) as executor:
            results = [executor.submit(self.startAutomation) for _ in range(1)]
            for f in concurrent.futures.as_completed(results):
                print(f.result())



        # calculate our total time elapsed
        end_time = time.perf_counter()

        time_elapsed = end_time - start_time
        print(f"Time elapsed is {round(time_elapsed, 2)}sec(s)")





class FileScreenerBot(QMainWindow):
    def __init__(self, widget):
        super(FileScreenerBot, self).__init__()
        self.setWindowTitle("Collation File Screener(Version 1.0.0)")

        # create a menu bar
        self.menu = self.menuBar()
        self.menu.setNativeMenuBar(False)
        self.file_menu = self.menu.addMenu("Menu")

        self.status = self.statusBar()
        self.status.showMessage("Application started")

        exit_action = QAction("Exit", self)
        exit_action.setShortcut("Ctrl+Q")    # Take note: no spacing in between the "Ctrl+Q" string
        exit_action.triggered.connect(self.exit_app)

        self.file_menu.addAction(exit_action)

        # setting the central widget
        self.setCentralWidget(widget)


    # signal/slot connection(element.signal_name.connect(slot_name))
    # the exit option must be connected to a slot that triggers the application to exit
    @Slot()
    def exit_app(self):
        QApplication.quit()






# now create an empty widget
class Widget(QWidget):
    def __init__(self):
        super(Widget, self).__init__()

        #######
        # left handside layout
        #***********************
        # setting the layout for top
        # The QVBoxLayout() provides the container to place widgets vertically
        self.left_top = QGroupBox("Settings")
        self.left_top.setMaximumHeight(90)



        # The QHBoxLayout() provides the container to place widgets horizontally
        self.proxy = QHBoxLayout()

        # selecting proxy
        # self.proxyLabel= QLabel("Proxy")
        self.proxyLabel= QLabel("")
        # self.proxyLabel.setMaximumWidth(70)

        # setting the settings dropdown
        self.combobox1 = QComboBox()
        self.combobox1.setMaximumWidth(130)
        self.combobox1.addItem("None")
        self.combobox1.addItem("Custom")
        self.combobox1.setDisabled(True)

        # adding the widgets to the QHBoxLayout()
        self.proxy.addWidget(self.proxyLabel)
        self.proxy.addWidget(self.combobox1)



        # Creating an overall QVBox to house thte two seperate horizontal components
        # The QHBoxLayout() provides the container to place widgets horizontally
        self.settingHandV = QVBoxLayout()
        self.settingHandV.addLayout(self.proxy)


        # then adding up to the groupbox
        self.left_top.setLayout(self.settingHandV)


        #***********************
        #***********************
        # setting the layout for the bottom left
        self.validPrompt = QLineEdit()
        self.validPrompt.setMaximumWidth(150)
        self.validPrompt.setMinimumHeight(300)
        self.validPrompt.setDisabled(True)

        #***********************
        # setting the layout for the overall left
        # The QVBoxLayout() provides the container to place widgets vertically
        self.layout_left = QVBoxLayout()
        self.layout_left.setMargin(1)
        

        # Adding the layout for overall left hand side
        # self.layout_left.addWidget(self.left_top)
        self.layout_left.addWidget(self.validPrompt)





        # #######
        # right handside layout
        #***********************
        # top-right handside layout
        # setting the layout
        # The QHBoxLayout() provides the container to place widgets horinzontally
        self.right_top = QHBoxLayout()
        self.right_top.setMargin(5)


        # # Adding our form layout
        # Set no. of threads
        self.fbox = QFormLayout()
        self.thread_countLabel = QLabel("Thread count")
        # self.thread_countLabel.setMaximumWidth(50)
        self.thread_countLabel.setStyleSheet("background-color: brown; color: white;")
        self.thread_countField = QLineEdit()
        self.thread_countField.setMaximumWidth(60)

        # Arrange form
        self.right_top_a = QHBoxLayout()
        # searchBox
        self.searchItemLabel = QLabel("Search Item")
        self.searchItemLabel.setMaximumWidth(70)
        # self.searchItemLabel.setMinimumWidth(100)

        self.searchItemField = QLineEdit()
        # self.searchItemField.setPlaceholderText(",,,")
        self.searchItemField.setMaximumWidth(150)




        self.right_top_b = QGridLayout()
        # self.right_top_b.setColumnStretch(0,0)
        # self.right_top_b.setColumnStretch(1,0)
        # self.right_top_b.setColumnStretch(2,0)
        self.right_top_b.setHorizontalSpacing(3)
        # self.right_top_b.setContentsMargins(0,0,0,3)

        self.right_top_b.addWidget(self.searchItemLabel, 0, 0)
        self.right_top_b.addWidget(self.searchItemField, 0, 1)

        self.radiobutton = QRadioButton("Contains")
        # self.radiobutton.setChecked(True)
        self.radiobutton.search_filter = "Contains"
        self.radiobutton.toggled.connect(self.onRadioClicked)
        self.right_top_b.addWidget(self.radiobutton, 0, 2)

        self.radiobutton = QRadioButton("Starts with")
        self.radiobutton.search_filter = "Starts with"
        self.radiobutton.toggled.connect(self.onRadioClicked)
        self.right_top_b.addWidget(self.radiobutton, 0, 3)

        self.radiobutton = QRadioButton("Ends with")
        self.radiobutton.search_filter = "Ends with"
        self.radiobutton.toggled.connect(self.onRadioClicked)
        self.right_top_b.addWidget(self.radiobutton, 0, 4)



        self.right_top_c = QHBoxLayout()
        # self.right_top_c.setMargin(5)

        # Root
        self.rootpathLabel = QLabel("Root Path")
        self.rootpathLabel.setMaximumWidth(55)
        self.rootpathField = QLineEdit()
        # self.rootpathField.setPlaceholderText("")
        self.rootpathField.setMaximumWidth(150)



        # Browse
        self.browse_file_button = QPushButton("Browse")
        self.browse_file_button.setMaximumWidth(70)
        self.browse_file_button.setStyleSheet("background-color: green; color: white;")
        self.browse_file_button.clicked.connect(self.getFiles)

        # Re-Index
        self.re_index_button = QPushButton("Re-Index")
        self.re_index_button.setMaximumWidth(70)
        self.re_index_button.setStyleSheet("background-color: green; color: white;")
        self.re_index_button.clicked.connect(self.re_indexFunc)

        # Search
        self.search_button = QPushButton("Search")
        self.search_button.setMaximumWidth(70)
        self.search_button.setStyleSheet("background-color: green; color: white;")
        self.search_button.clicked.connect(self.searchFunc)

        self.right_top_c.addWidget(self.rootpathLabel)
        self.right_top_c.addWidget(self.rootpathField)
        self.right_top_c.addWidget(self.browse_file_button)
        self.right_top_c.addWidget(self.re_index_button)
        self.right_top_c.addWidget(self.search_button)






        self.fbox.addRow(self.thread_countLabel, self.thread_countField)
        self.fbox.addRow(self.right_top_b)

        self.fbox.addRow(self.right_top_c)
        # self.fbox.addRow(self.rootpathLabel, self.right_top_c)





        






        #***********************
        # bottom-right handside layout
        # setting the layout
        # The QVBoxLayout() provides the container to place widgets vertically
        self.right_bottom = QVBoxLayout()
        self.right_bottom.setMargin(5)

        # label progress bar
        self.progress_label = QLabel("Progress")

        # Adding the progress Bar
        self.progress = QProgressBar()
        self.progress.setMaximumWidth(750)
        self.progress.setStyleSheet("margin-bottom: 5px; text-align: center;")
        self.progress.setMaximum(N)

        # Adding the debugger label
        self.l2 = QLabel()	
        self.l2.setText("Log Messages")

        # Debuging box
        self.debugging = QTextEdit()
        self.debugging.setReadOnly(True)
        self.debugging.setMaximumWidth(750)
        self.debugging.setMinimumHeight(300)
        self.debugging.setStyleSheet("background-color: black; color: white; border: 1px solid;")


        # ************************************
        # The QHBoxLayout() provides the container to place widgets horinzontally
        self.right_bottom_last = QHBoxLayout()

        # Set start button
        self.start_button = QPushButton("Start")
        self.start_button.setMaximumWidth(70)
        self.start_button.setStyleSheet("background-color: green; color: white;")
        # self.start_button.clicked.connect(self.startFunc)
        self.start_button.clicked.connect(self.getFiles)

        # Set stop button
        self.stop_button = QPushButton("Stop")
        self.stop_button.setStyleSheet("background-color: grey; color: white;")
        self.stop_button.setMaximumWidth(70)
        self.stop_button.setDisabled(True)

        # Clear screen button
        # Set clear button
        self.clear_button = QPushButton("Clear Screen")
        self.clear_button.setMaximumWidth(70)
        self.clear_button.setStyleSheet("background-color: purple; color: white;")
        self.clear_button.clicked.connect(self.clearFunc)


        # Adding the layouts
        self.right_top.addLayout(self.fbox)

        self.right_bottom.addWidget(self.progress_label)
        self.right_bottom.addWidget(self.progress)
        self.right_bottom.addWidget(self.l2)
        self.right_bottom.addWidget(self.debugging)

        # self.right_bottom_last.addWidget(self.start_button)
        self.right_bottom_last.addWidget(self.stop_button)
        self.right_bottom_last.addWidget(self.clear_button)


        #***********************
        # Setting the layout for the right hand side
        # QWidget layout
        # the QVBoxLayout() provides the container to place widgets vertically
        self.layout_right = QVBoxLayout()
        self.layout_right.addLayout(self.right_top)
        # self.layout_right.addLayout(self.right_top_c)
        self.layout_right.addLayout(self.right_bottom)
        self.layout_right.addLayout(self.right_bottom_last)


        
        # ##########
        # Setting the overall layout
        # QWidget layout
        # the QHBoxLayout() provides the container to place widgets horizontally
        self.layout_overall = QHBoxLayout()

        # self.table_view.setSizePolicy(size)
        self.layout_overall.addLayout(self.layout_left)
        self.layout_overall.addLayout(self.layout_right)

        # set the layout to the QWidget
        self.setLayout(self.layout_overall)


    @Slot()
    def re_indexFunc(self):
        pass


    @Slot()
    def searchFunc(self):
        # Start traversing selected directory

        # setting our variables
        # global thread_count, proxy_usage, authProxy, rangeValue
        global proxy_usage, thread_count, file_search_path, search_keyword
        proxy_usage = self.combobox1.currentText()
        thread_count = self.thread_countField.text()
        file_search_path = self.rootpathField.text()
        search_keyword = self.searchItemField.text()

        # Setting the value on every run to 0
        self.progress.setValue(0)

        # Creating the instance of our thread
        self.get_thread = ThreadingClass()

        # Next we need to connect the events from that thread to functions we want
        # to run when those signals get fired
        
        # Updating the progress will be handeled in the update_progress method and the signal that
        # the thread will emit is SIGNAL("update_progress(QString)") which is a custom signal
        # the rest is same as we can use to connect any signal
        # self.connect(self.get_thread, SIGNAL("update_progress(QString)"), self.update_progress)
        self.connect(self.get_thread, SIGNAL("update_progress()"), self.update_progress)
        self.connect(self.get_thread, SIGNAL("update_alert(QString)"), self.update_alert)

        # This is pretty self explanatory
        # regardless of whether the thread finishes or the user terminates it
        # we want to show the notification to the user that adding is done
        # and regardless of whether it was terminated or finished by itself
        # the finished signal will go off. So we don't need to catch the
        # terminated one specifically, but we could if we wanted.
        # In essence, the signal that the thread will emit is SIGNAL("finished()")
        # Which is an inbuilt signal
        self.connect(self.get_thread, SIGNAL("finished()"), self.done)

        # We have all the events we need connected we can start the thread
        # starting our thread
        self.get_thread.start()

        # Then change buttons appearances
        # Make change for the start button
        self.search_button.setEnabled(False)
        self.search_button.setStyleSheet("background-color: grey; color: white;")


        # Make change for the stop button
        self.stop_button.setEnabled(True)
        self.stop_button.setStyleSheet("background-color: red; color: white;")


        # Make change for the clear button
        self.clear_button.setEnabled(False)
        self.clear_button.setStyleSheet("background-color: grey; color: white;")

        # make the connections
        self.stop_button.clicked.connect(self.get_thread.terminate)
        self.stop_button.clicked.connect(self.done)




    @Slot()
    def onRadioClicked(self):
        global search_method
        self.radio_button = self.sender()
        if self.radio_button.isChecked():
            search_method = self.radio_button.search_filter
            # print("Search method is %s" % (self.radio_button.search_filter))


    @Slot()
    def getFiles(self):
        self.dlg = QFileDialog()
        self.dlg.setDirectory(base_dir)
        self.dlg.setViewMode(QFileDialog.Detail)
        current_directory = self.dlg.getExistingDirectory()
        # print(f"Selected directory: {current_directory}")   # Returns a list of file paths
        self.rootpathField.setText(current_directory)


    @Slot()
    def clearFunc(self):
        # clear our screen
        self.debugging.clear()


    @Slot()
    def startFunc(self):
        # setting our variables
        # global thread_count, proxy_usage, authProxy, rangeValue
        global proxy_usage, thread_count, username, password
        proxy_usage = self.combobox1.currentText()
        thread_count = self.thread_countField.text()
        username = self.usernameField.text()
        password = self.passwordField.text()


        # Setting the value on every run to 0
        self.progress.setValue(0)

        # Creating the instance of our thread
        self.get_thread = ThreadingClass()

        # Next we need to connect the events from that thread to functions we want
        # to run when those signals get fired
        
        # Updating the progress will be handeled in the update_progress method and the signal that
        # the thread will emit is SIGNAL("update_progress(QString)") which is a custom signal
        # the rest is same as we can use to connect any signal
        # self.connect(self.get_thread, SIGNAL("update_progress(QString)"), self.update_progress)
        self.connect(self.get_thread, SIGNAL("update_progress()"), self.update_progress)

        # This is pretty self explanatory
        # regardless of whether the thread finishes or the user terminates it
        # we want to show the notification to the user that adding is done
        # and regardless of whether it was terminated or finished by itself
        # the finished signal will go off. So we don't need to catch the
        # terminated one specifically, but we could if we wanted.
        # In essence, the signal that the thread will emit is SIGNAL("finished()")
        # Which is an inbuilt signal
        self.connect(self.get_thread, SIGNAL("finished()"), self.done)

        # We have all the events we need connected we can start the thread
        # starting our thread
        self.get_thread.start()

        # Then change buttons appearances
        # Make change for the start button
        self.start_button.setEnabled(False)
        self.start_button.setStyleSheet("background-color: grey; color: white;")


        # Make change for the stop button
        self.stop_button.setEnabled(True)
        self.stop_button.setStyleSheet("background-color: red; color: white;")


        # Make change for the clear button
        self.clear_button.setEnabled(False)
        self.clear_button.setStyleSheet("background-color: grey; color: white;")

        # make the connections
        self.stop_button.clicked.connect(self.get_thread.terminate)
        self.stop_button.clicked.connect(self.done)
        

    @Slot()
    # def update_progress(self, post_text):
    def update_progress(self):
        """
        Add the text that's given to this function to the
        list_submissions QListWidget we have in our GUI and
        increase the current value of progress bar by 1

        :param post_text: text of the item to add to the list
        :type post_text: str
        """
        # self.validPrompt.insert(post_text)
        self.progress.setValue(self.progress.value()+1)


    @Slot()
    def done(self):
        """
        Show the message that fetching posts is done.
        Disable Stop button, enable the Start one and reset progress bar to 0
        """
        self.stop_button.setEnabled(False)
        self.stop_button.setStyleSheet("background-color: grey; color: white;")
        self.start_button.setEnabled(True)
        self.start_button.setStyleSheet("background-color: green; color: white;")
        self.clear_button.setEnabled(True)
        self.clear_button.setStyleSheet("background-color: purple; color: white;")
        self.search_button.setEnabled(True)
        self.search_button.setStyleSheet("background-color: green; color: white;")

        self.progress.setValue(0)

    @Slot(str)
    def update_alert(self, keyWordString_string):
        """
        Update and change from signal to current screen
        """
        msgBox = QMessageBox()
        msgBox.setStyleSheet("background-color: purple; color: white;")
        msgBox.setIcon(QMessageBox.Information)
        msgBox.setText("Match Found!")
        msgBox.setInformativeText("Match Found For keyword:. '%s'" % keyWordString_string)
        msgBox.setWindowTitle("Message Alert Window")
        # msgBox.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
        msgBox.exec_()



    @Slot(str)
    def append_text(self, text):
        self.debugging.moveCursor(QTextCursor.End)
        self.debugging.insertPlainText(text)



# Create Queue and redirect sys.stdout and sys.stderr to this queue
queue = Queue()
sys.stdout = WriteStream(queue)
sys.stderr = WriteStream(queue)        



if __name__ == "__main__":
    app = QApplication(sys.argv)
    # using QWidget as the central widget
    widget = Widget()
    window = FileScreenerBot(widget)
    window.resize(1000, 600)
    window.show()    

    # Create a thread that will listen on the other end of the queue, 
    # and send the text to the debugging QTextEdit() widget in our application
    thread = QThread()
    my_receiver = MyReceiver(queue)
    my_receiver.mysignal.connect(widget.append_text)
    my_receiver.moveToThread(thread)
    thread.started.connect(my_receiver.run)
    thread.start()
    sys.exit(app.exec_())












